In [6]:
import numpy as nm
!pip install np_utils

  Created wheel for np-utils: filename=np_utils-0.5.12.1-cp37-none-any.whl size=57126 sha256=c77cf2bdf933f848435a88ecd6771509991aa3b959fce3d08ceb915e1fba224b
  Stored in directory: C:\Users\ASUS\AppData\Local\pip\Cache\wheels\92\4b\81\206efd0d01330a96f3aebe5021d2d5f0b264b7ade827c306ef
Successfully built np-utils


In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import pandas

from numpy import mean
from numpy import std
from numpy import dstack

from keras.layers import Flatten
from keras.layers import Dropout

from keras.utils import to_categorical
from matplotlib import pyplot


ImportError: cannot import name 'Sequence' from 'keras.utils' (C:\Users\ASUS\Anaconda3\lib\site-packages\keras\utils\__init__.py)

In [5]:
dataframe = pandas.read_csv("ALLfeature.csv", header=None)
dataset = dataframe.values
print(dataset)


[['accmeanx' 'accstdx' 'accmaxx' ... 'accsma' 'gyrosma' 'activity']
 ['-1.92301239894' '0.08970598327798304' '-1.685439567' ...
  '13.92218831748' '0.05290931311599999' 'Exercise']
 ['-1.89956705936' '0.05744322654161099' '-1.797349574' ...
  '13.915354729040002' '0.009526680536' 'Exercise']
 ...
 ['-1.76223887468' '1.4788593308555815' '0.579155463' ...
  '13.128671348579994' '2.18813961348' 'Walkingupstair']
 ['-1.61768404854' '1.7910274717745818' '1.4302321180000002' ...
  '14.156987563180001' '2.4643959640600004' 'Walkingupstair']
 ['-1.1109272277800002' '1.4077288909778327' '1.4302321180000002' ...
  '12.561446300180005' '2.0246994448399995' 'Walkingupstair']]


In [6]:
X = dataset[1:,:-1].astype(float)
Y = dataset[1:,-1]
print(Y)


['Exercise' 'Exercise' 'Exercise' ... 'Walkingupstair' 'Walkingupstair'
 'Walkingupstair']


In [7]:
Y

array(['Exercise', 'Exercise', 'Exercise', ..., 'Walkingupstair',
       'Walkingupstair', 'Walkingupstair'], dtype=object)

In [8]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)

In [9]:
dummy_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [10]:
from sklearn.model_selection import train_test_split


In [11]:
X_train,X_test, y_train,y_test=train_test_split(X,dummy_y,test_size=0.2,random_state=42)

In [15]:
X_train=dstack(X_train)
X_train.shape

(3421, 1, 50)

In [17]:
X_test=dstack(X_test)
X_test.shape

(856, 1, 50)

In [18]:
y_train.shape
# y_train.shape

(3421, 10)

In [19]:
# y_test=dstack(y_test)
y_test.shape

(856, 10)

In [20]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [21]:
# fit and evaluate a model
def evaluate_model(X_train, y_train, X_test, y_test):
    verbose, epochs, batch_size = 0, 50, 32
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    model = Sequential()
    model.add(LSTM(10, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _,accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    
    
    return accuracy

In [22]:
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=5):
    # load data
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(X_train, y_train, X_test, y_test)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [23]:
run_experiment()




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
>#1: 75.234
>#2: 74.416
>#3: 94.509
>#4: 81.776
>#5: 94.276
[75.23364485981308, 74.41588785046729, 94.5093458500978, 81.77570087887418, 94.27570093457945]
Accuracy: 84.042% (+/-8.828)


In [24]:
def evaluate_mo(X_train, y_train, X_test, y_test):
    verbose, epochs, batch_size = 0, 50, 32
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    model = Sequential()
    model.add(LSTM(10, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [25]:
model=evaluate_mo(X_train, y_train, X_test, y_test)
model.fit(X_train, y_train, batch_size=32, epochs=30)

Epoch 1/30
3421/3421 [==============================] - 1s - loss: 2.3299 - acc: 0.1684     
Epoch 2/30
3421/3421 [==============================] - 0s - loss: 2.0850 - acc: 0.2815     
Epoch 3/30
3421/3421 [==============================] - 0s - loss: 1.8990 - acc: 0.3709     
Epoch 4/30
3421/3421 [==============================] - 0s - loss: 1.7348 - acc: 0.4136     
Epoch 5/30
3421/3421 [==============================] - 0s - loss: 1.6140 - acc: 0.4475     
Epoch 6/30
3421/3421 [==============================] - 0s - loss: 1.5015 - acc: 0.4835     
Epoch 7/30
3421/3421 [==============================] - 0s - loss: 1.4414 - acc: 0.5013     
Epoch 8/30
3421/3421 [==============================] - 0s - loss: 1.3896 - acc: 0.5244     
Epoch 9/30
3421/3421 [==============================] - 0s - loss: 1.3501 - acc: 0.5300     
Epoch 10/30
3421/3421 [==============================] - 0s - loss: 1.3230 - acc: 0.5300     
Epoch 11/30
3421/3421 [==============================] - 0s - loss: 1

In [26]:
import shap


In [27]:
explainer = shap.DeepExplainer(model, X_train)
# explain the the testing instances (can use fewer instanaces)
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(X_test)
# init the JS visualization code
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], features)

keras is no longer supported, please use tf.keras instead.


AttributeError: 'Dense' object has no attribute '_inbound_nodes'